<h1><span style="color:red">Generate Contingency Tables</span></h1>

### This notebook reads numeric and categorical variables from the survey dataset, lets users compute a contingency table from variables of interest and test for independence.

## 1. Retrieve survey parameters from the URL

In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

In [2]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import datetime
import pandas as pd    
import numpy as np
import panel as pn
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "/home/jovyan/jupyter-suave/temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

## 2. Read the survey file

In [3]:
# read the csv file
df = panellibs.extract_data(absolutePath + csv_file)

# create a list of variable names
variables_df = pd.DataFrame({'varname':df.columns})
printmd("<b><span style='color:red'>All variables in the survey file:</span></b>")
print(variables_df.varname.values)

<b><span style='color:red'>All variables in the survey file:</span></b>

['Name' 'OAID#link#multi' 'Affiliation#sortquan' 'City#sortquan'
 'Region#sortquan' 'Country#sortquan' 'Latitude#hidden' 'Longitude#hidden'
 'Collaborators#multi#link#sortquan' 'Scope#multi#sortquan'
 'Keywords#multi#sortquan' 'OA concepts#multi#sortquan'
 'Publications#hidden' 'Publication Dates#multi#sortquan' '#img' '#netvis']


## 3. Select variables from survey

In [4]:
# select number of variables for contingency table
n_row = pn.widgets.IntSlider(name='Select Number of Row Variables',start=1, end=2, value=1)
n_col = pn.widgets.IntSlider(name='Select Number of Column Variables',start=1, end=2, value=1)

pn.Column(n_row, n_col)

Column
    [0] IntSlider(end=2, name='Select Number o..., start=1, value=1)
    [1] IntSlider(end=2, name='Select Number o..., start=1, value=1)

In [5]:
# select variables from survey
row_vars = pn.widgets.MultiChoice(name='Select Row Variables for Contingency Table',
                                  value=[], options=list(df.columns), max_items=n_row.value)
col_vars = pn.widgets.MultiChoice(name='Select Column Variables for Contingency Table',
                                  value=[], options=list(df.columns), max_items=n_col.value)
pn.Row(row_vars, col_vars, height=350)

Row(height=350)
    [0] MultiChoice(max_items=1, name='Select Row Variables f..., options=['Name', 'OAID#link#multi'...])
    [1] MultiChoice(max_items=2, name='Select Column V..., options=['Name', 'OAID#link#multi'...])

## 4. Bin Numerical and Date Variables

Create custom bins for numerical and date variables, or skip to the next step.

<h3><span style="color:red">(OPTIONAL) Set custom bins for numerical variables:</span></h3>

In [6]:
# select numerical columns to bin -- **unselected columns will be automatically binned into 5 equal bins**
df = df.apply(pd.to_numeric, errors='ignore')
df = df.dropna(axis=1, how='all')
op = [i for i in list(df.columns) if '#number' in i]
binnable = pn.widgets.MultiChoice(name='Select Columns to Bin: ', options=op, height=350)
binnable

MultiChoice(height=350, name='Select Columns to Bin: ')

In [7]:
# set custom bins for a numeric (#number) variable
next_var = pn.widgets.Button(name='Continue to Next Variable', button_type='success')
first_col = binnable.value[next_var.clicks]
selected_bins = [[] for i in binnable.value]

text = pn.widgets.TextInput(name='Current Variable: ', value=first_col, disabled=True)
num = pn.widgets.Select(name='Select number of bins', options=[i+1 for i in range(5)])
next_bin = pn.widgets.Button(name='Next Bin', button_type='primary')
bin_range = pn.widgets.RangeSlider(name='Select Range for Bin #1',
                                   start=df[first_col].min(), end=df[first_col].max(),
                                   value=(df[first_col].min(), df[first_col].max()), step=2)

def plot_histogram(df, column, plotting_pane, x_range=None):
    """
    Helper function to plot histogram of a numeric variable
    in the provided x_range onto the panel plotting pane.
    """
    fig, ax = plt.subplots(1,1)
    df[column].plot.hist(bins=50, ax=ax, title = 'Histogram of: ' + column, xlim=x_range)
    ax.set_xlabel(column)
    plotting_pane.object = fig
    plt.close()

def b(event):
    """
    Updates bin sliders when "next" is clicked
    """
    if next_bin.clicks == 0:
        return
    
    selected_col = binnable.value[next_var.clicks]

    if next_bin.clicks < num.value:
        bin_range.name = 'Select Range for Bin #' + str(next_bin.clicks + 1)
        selected_bins[next_var.clicks].append(bin_range.value)
        bin_range.start = bin_range.value[1]
        bin_range.value = (bin_range.value[1], df[selected_col].max())
        plot_histogram(df, selected_col, plot, x_range = bin_range.value)
    else:
        selected_bins[next_var.clicks].append(bin_range.value)
        next_bin.disabled = True
        
        if selected_col != binnable.value[-1]:
            layout[0][4] = next_var
        else:
            layout[0][4] = "Variable Binning Complete!"

def c(event):
    """
    Resets layout when "continue" is clicked
    """
    next_bin.clicks = 0
    next_col = binnable.value[next_var.clicks] 
    text.value = next_col
    num.value = 1
    bin_range.start = df[next_col].min()
    bin_range.end = df[next_col].max()
    bin_range.value = (df[next_col].min(), df[next_col].max())
    next_bin.disabled = False
    layout[0][4] = ""
    plot_histogram(df, next_col, plot)

next_bin.on_click(b)
next_var.on_click(c)
plot =  pn.pane.Matplotlib(dpi=80)
plot_histogram(df, binnable.value[next_var.clicks], plot)
layout = pn.Row(pn.Column(text, num, bin_range, next_bin, ""), plot)
layout

IndexError: list index out of range

In [8]:
# bin each column with chosen bins
cols = binnable.value
for col, bins in list(zip(cols, selected_bins)):
    bins = pd.IntervalIndex.from_tuples(bins, closed='left')
    df[col] = pd.cut(df[col], bins)
    
# bin remaining numeric columns into 5 equal bins
remainder = list(set(binnable.options) - set(binnable.value))
df[remainder] = df[remainder].apply(pd.cut, bins=5)

NameError: name 'selected_bins' is not defined

<h3><span style="color:red">(OPTIONAL) Set custom bins for date variables:</span></h3>

In [7]:
# convert date variables in survey
dates = [i for i in list(df.columns) if '#date' in i]
df[dates] = df[dates].apply(pd.to_datetime, errors='coerce')
df[dates] = df[dates].apply(lambda x: x.dt.date)
d_max = pd.Series(df[dates].to_numpy().flatten()).dropna().max()
d_min = pd.Series(df[dates].to_numpy().flatten()).dropna().min()

# define bins for all date variables
date_bins = []
date_slider = pn.widgets.DateRangeSlider(name='Date Range for Bin #1 ', start=d_min, end=d_max, value=(d_min,d_max))
next_bin2 = pn.widgets.Button(name='Next Bin', button_type='primary')
num = pn.widgets.Select(name='Select number of bins', options=[i+1 for i in range(5)])

def d(event):
    """
    Updates bin sliders when "next" is clicked
    """
    if next_bin2.clicks < num.value:
        date_slider.name = 'Date Range for Bin #' + str(next_bin2.clicks + 1)
        date_bins.append(date_slider.value)
        date_slider.start = date_slider.value[1]
        date_slider.value = (date_slider.value[1], datetime.datetime(d_max.year, d_max.month, d_max.day))
        plot_dates(df, plot)
    else:
        date_bins.append(date_slider.value)
        next_bin2.disabled = True
        layout2[0][3] = 'Date Binning Complete!'
        
def plot_dates(df, plotting_pane):
    """
    Plots dates based on slider selection to the plotting pane.
    """
    all_dates = pd.DataFrame(df[dates].to_numpy().flatten(), columns=['date'])
    filtered = all_dates[(all_dates['date'] >= pd.Timestamp(date_slider.value[0]))]
    if len(filtered) == 0:
        fig, ax = fig, ax = plt.subplots(1,1)
        all_dates.groupby('date').size().plot(kind='bar', ax=ax)
        num = 20
    else:
        fig, ax = fig, ax = plt.subplots(1,1)
        filtered.groupby('date').size().plot(kind='bar', ax=ax)
        if len(filtered.groupby('date').size()) > 20:
            num = int(len(filtered.groupby('date').size())/15)
        else:
            num = 1
    ax.set_xticks(ax.get_xticks()[::num])
    ax.set_ylabel('Frequency');
    for tick in ax.get_xticklabels():
        tick.set_rotation(70)
    plotting_pane.object = fig
    plt.close()

next_bin2.on_click(d)
plot =  pn.pane.Matplotlib(dpi=80, tight=True)
plot_dates(df, plot)
layout2 = pn.Row(pn.Column(num, date_slider, next_bin2, ""), plot)
layout2

ValueError: Date parameter 'start' only takes datetime and date types, not type <class 'float'>.

In [ ]:
# convert selected bins to binning intervals
bins = pd.interval_range(start=pd.Timestamp(datetime.date(date_bins[0][0].year, date_bins[0][0].month, date_bins[0][0].day)),
                         end=pd.Timestamp(datetime.date(date_bins[0][1].year, date_bins[0][1].month, date_bins[0][1].day)),
                         periods=1)

for d_bin in date_bins[1:]:
    out = (datetime.date(d_bin[0].year, d_bin[0].month, d_bin[0].day),
           datetime.date(d_bin[1].year, d_bin[1].month, d_bin[1].day))
    interval = pd.interval_range(start=pd.Timestamp(out[0]), end=pd.Timestamp(out[1]), periods=1)
    bins = bins.append(interval)
    
# convert dates to date ranges
for d in dates:
    df[d] = df[d].apply(pd.Timestamp)
    df[d] = pd.cut(df[d], bins=bins)

## 4. Generate contingency table

In [8]:
def generate_table(df, row_variables, column_variables):
    """
    Helper function to generate a contingency table from the input dataframe
    with a given list of row_variables and column_variables.
    """
    tab = pd.crosstab([df[row_variables[i]] for i in range(len(row_variables))],
                      [df[column_variables[i]] for i in range(len(column_variables))])
    table = sm.stats.Table(tab)
    return table.table_orig, table

In [9]:
# generate contingency table
df['freq'] = 1
selected_row_vars = row_vars.value
selected_col_vars = col_vars.value
tab = generate_table(df, selected_row_vars, selected_col_vars)
tab[0]

Scope#multi#sortquan    aquifer                                               
Latitude#hidden      -24.666668 -22.551643  24.370000  30.363500  32.014885   
Longitude#hidden                                                              
-157.813830                   0          0          0          0          0  \
-123.311750                   0          0          0          0          0   
-123.279465                   0          0          0          0          0   
-123.246080                   0          0          0          0          0   
-122.636490                   0          0          0          0          0   
...                         ...        ...        ...        ...        ...   
 146.758330                   0          0          0          0          0   
 149.124860                   0          0          0          0          0   
 151.932000                   0          0          0          0          0   
 153.022860                   0          0          0          0          0   
 172.642360                   0          0          0          0          0   

Scope#multi#sortquan                                                          
Latitude#hidden       32.848080  33.209440  34.157010  38.318336  38.947620   
Longitude#hidden                                                              
-157.813830                   0          0          0          0          0  \
-123.311750                   0          0          0          0          0   
-123.279465                   0          0          0          0          0   
-123.246080                   0          0          0          0          0   
-122.636490                   0          0          0          0          0   
...                         ...        ...        ...        ...        ...   
 146.758330                   0          0          0          0          0   
 149.124860                   0          0          0          0          0   
 151.932000                   0          0          0          0          0   
 153.022860                   0          0          0          0          0   
 172.642360                   0          0          0          0          0   

Scope#multi#sortquan  ... aquifer|transboundary                         
Latitude#hidden       ...             56.950980  56.963486  57.688330   
Longitude#hidden      ...                                               
-157.813830           ...                     0          0          0  \
-123.311750           ...                     0          0          0   
-123.279465           ...                     0          0          0   
-123.246080           ...                     0          0          0   
-122.636490           ...                     0          0          0   
...                   ...                   ...        ...        ...   
 146.758330           ...                     0          0          0   
 149.124860           ...                     0          0          0   
 151.932000           ...                     0          0          0   
 153.022860           ...                     0          0          0   
 172.642360           ...                     0          0          0   

Scope#multi#sortquan                                                          
Latitude#hidden       57.698345  59.335155  59.402410  59.438686  59.930088   
Longitude#hidden                                                              
-157.813830                   0          0          0          0          0  \
-123.311750                   0          0          0          0          0   
-123.279465                   0          0          0          0          0   
-123.246080                   0          0          0          0          0   
-122.636490                   0          0          0          0          0   
...                         ...        ...        ...        ...        ...   
 146.758330                   0          0          

In [10]:
# denote shape of contingency table as two-way, three-way, or multi-way
table_shape = None
total_vars = len(row_vars.value + col_vars.value)
if total_vars == 2:
    table_shape = 'two-way'
elif total_vars == 3:
    table_shape = 'three-way'
else:
    table_shape = 'multi-way'
table_shape

'three-way'

## 5. Test for mutual independence

For one-way tables conduct a chi-square test of independence, under the null hypothesis that the two variables of interest are independent. For three-way and multi-way tables, fit a log-linear model to test for mutual independence between all variables of interest.  

In [11]:
def chi_square(table, alpha=.05):
    """
    Performs a chi-square test of independence in a
    two-way contingency table at the given significance level
    """
    chi2 = table.test_nominal_association()
    
    # display a warning if expected counts are < 5
    exp_freq = table.fittedvalues.to_numpy()
    if (exp_freq < 5).sum() != 0:
        print("Warning: table contains expected frequencies less than 5.")

    # display results of chi-square test
    p_val = chi2.pvalue
    if p_val < alpha:
        print("P-value: {}. Table variables are associated at significance level: {}".format(p_val.round(4), alpha))
    else:
        print("P-value: {}. Table variables are independent at significance level: {}".format(p_val.round(4), alpha))
        
        
def llm_mutual(flat_table):
    """
    Performs log-linear analysis of the flattened multi-way
    contingency table using Poisson regression for mutual independence.
    """
    # remove suave tags from column names (interferes with statsmodels api)
    df_2 = flat_table.copy()
    df_2.columns = [i.split('#')[0].replace(' ', '_') for i in flat_table.columns]
    
    # define predictor and response variables
    predictors = (' + ').join([i.split('#')[0].replace(' ', '_') for i in row_vars.value + col_vars.value])
    response = 'freq'
    
    # fit model
    glm = smf.glm(response + ' ~ ' + predictors, data=df_2, family=sm.families.Poisson())
    res_o = glm.fit()
    print(res_o.summary())
    return res_o

In [12]:
# perform chi-square test of independence for two-way tables
if table_shape == 'two-way':
    result = chi_square(tab[1])
# perform log-linear analysis for three-way and multi-way tables
else:
    flat = df.groupby(col_vars.value + row_vars.value).sum()['freq'].to_frame().reset_index()
    result = llm_mutual(flat)
result

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   freq   No. Observations:                  462
Model:                            GLM   Df Residuals:                      458
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -836.32
Date:                Tue, 12 Sep 2023   Deviance:                       572.99
Time:                        22:37:21   Pearson chi2:                     877.
No. Iterations:                     5   Pseudo R-squ. (CS):           0.003460
Covariance Type:            nonrobust                                         
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

<h2><span style="color:red">5a. Optional: Test for joint independence</span></h2>

<span style="color:red">The following log-linear models are for testing THREE-WAY tables only!</span> 

For three-way tables, test variables for joint-independence. The log-linear model tests for joint independence under the assumption that a single variable of interest is independent of the other two. The model implies that other two variables can have an arbitrary association, and that this association doesn't depend on the level of the variable of interest.

In [13]:
# select a variable to test for joint indpendence
selector = pn.widgets.Select(name='Test joint indpedence of: ', options=col_vars.value + row_vars.value)
selector

Select(name='Test joint indpedence o..., options=['Scope#multi#sortquan', ...], value='Scope#multi#sortquan')

In [14]:
def llm_joint(flat_table):
    """
    Performs log-linear analysis of the flattened multi-way
    contingency table using Poisson regression for joint independence.
    """
    # remove suave tags from column names (interferes with statsmodels api)
    df_2 = flat_table.copy()
    df_2.columns = [i.split('#')[0].replace(' ', '_') for i in flat_table.columns]
    
    # define predictor and response variables
    var = row_vars.value + col_vars.value
    var.remove(selector.value)
    selected_var = selector.value.split('#')[0].replace(' ', '_')
    predictors = [i.split('#')[0].replace(' ', '_') for i in var]
    formula = 'freq ~ ' + selected_var + ' + ' + predictors[0] + ' + ' + predictors[1] + ' + ' + \
                predictors[0] + ' * ' + predictors[1]
    print('Model: ' + formula)
    
    # fit model
    glm = smf.glm(formula, data=df_2, family=sm.families.Poisson())
    res_o = glm.fit()
    return res_o

In [15]:
if table_shape == 'three-way':
    flat = df.groupby(col_vars.value + row_vars.value).sum()['freq'].to_frame().reset_index()
    result = llm_joint(flat)
result.summary()

Model: freq ~ Scope + Longitude + Latitude + Longitude * Latitude


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   freq   No. Observations:                  462
Model:                            GLM   Df Residuals:                      457
Model Family:                 Poisson   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -835.80
Date:                Tue, 12 Sep 2023   Deviance:                       571.96
Time:                        22:37:28   Pearson chi2:                     873.
No. Iterations:                     5   Pseudo R-squ. (CS):           0.005690
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          0.5234      0.177      2.963      0.003       0.177       0.870
Scope[T.aquifer|transboundary]     0.1785      0.173      1.034      0.301      -0.160       0.517
Longitude                         -0.0002      0.001     -0.279      0.780      -0.002       0.001
Latitude                          -0.0003      0.001     -0.197      0.843      -0.003       0.003
Longitude:Latitude              2.199e-05   2.19e-05      1.003      0.316    -2.1e-05    6.49e-05
==================================================================================================
"""

<h2><span style="color:red">5b. Optional: Test for conditional independence</span></h2>

For three-way tables, test variables for conditional independence. The log-linear model tests for conditional indpendence under the asumption that two variables are independent, given the third.

In [16]:
# select a variable to condition the independence of the remaining variables
selector = pn.widgets.Select(name='Test if variables are indepndent given: ', options=col_vars.value + row_vars.value)
selector

Select(name='Test if variables a..., options=['Scope#multi#sortquan', ...], value='Scope#multi#sortquan')

In [17]:
def llm_conditional(flat_table):
    """
    Performs log-linear analysis of the flattened multi-way
    contingency table using Poisson regression for conditional independence.
    """
    # remove suave tags from column names (interferes with statsmodels api)
    df_2 = flat_table.copy()
    df_2.columns = [i.split('#')[0].replace(' ', '_') for i in flat_table.columns]
    
    # define predictor and response variables
    var = row_vars.value + col_vars.value
    var.remove(selector.value)
    selected_var = selector.value.split('#')[0].replace(' ', '_')
    predictors = [i.split('#')[0].replace(' ', '_') for i in var]
    formula = 'freq ~ ' + selected_var + ' + ' + predictors[0] + ' + ' + predictors[1] + ' + ' + \
                predictors[0] + ' * ' + selected_var + '+ ' + predictors[1] +  ' * ' + selected_var
    print('Model: ' + formula)
    
    # fit model
    glm = smf.glm(formula, data=df_2, family=sm.families.Poisson())
    res_o = glm.fit()
    return res_o

In [18]:
if table_shape == 'three-way':
    flat = df.groupby(col_vars.value + row_vars.value).sum()['freq'].to_frame().reset_index()
    result = llm_conditional(flat)
result.summary()

Model: freq ~ Scope + Longitude + Latitude + Longitude * Scope+ Latitude * Scope


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   freq   No. Observations:                  462
Model:                            GLM   Df Residuals:                      456
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -835.95
Date:                Tue, 12 Sep 2023   Deviance:                       572.25
Time:                        22:37:31   Pearson chi2:                     877.
No. Iterations:                     5   Pseudo R-squ. (CS):           0.005061
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    0.2477      0.396      0.625      0.532      -0.529       1.025
Scope[T.aquifer|transboundary]               0.4539      0.401      1.133      0.257      -0.331       1.239
Longitude                                    0.0013      0.002      0.549      0.583      -0.003       0.006
Longitude:Scope[T.aquifer|transboundary]    -0.0009      0.002     -0.397      0.691      -0.006       0.004
Latitude                                     0.0069      0.009      0.742      0.458      -0.011       0.025
Latitude:Scope[T.aquifer|transboundary]     -0.0074      0.009     -0.778      0.437      -0.026       0.011
============================================================================================================
"""

<h2><span style="color:red">5c. Optional: Test for homogeneous association</span></h2>

For three-way tables, test for homogeneous association between the variables. The log-linear model, or no three-factor interaction model, tests for homogeneous association and implies that the conditional reelationship between any pair of variables given the third is the same at each level of the third variable.

In [19]:
def llm_association(flat_table):
    """
    Performs log-linear analysis of the flattened multi-way
    contingency table using Poisson regression for homogeneous association.
    """
    # remove suave tags from column names (interferes with statsmodels api)
    df_2 = flat_table.copy()
    df_2.columns = [i.split('#')[0].replace(' ', '_') for i in flat_table.columns]
    
    # define predictor and response variables
    predictors = (' + ').join([i.split('#')[0].replace(' ', '_') for i in row_vars.value + col_vars.value])
    formula = 'freq ~ (' + predictors + ')**2'
    print('Model: ' + formula)
    
    # fit model
    glm = smf.glm(formula, data=df_2, family=sm.families.Poisson())
    res_o = glm.fit()
    return res_o

In [20]:
if table_shape == 'three-way':
    flat = df.groupby(col_vars.value + row_vars.value).sum()['freq'].to_frame().reset_index()
    result = llm_association(flat)
result.summary()

Model: freq ~ (Longitude + Scope + Latitude)**2


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   freq   No. Observations:                  462
Model:                            GLM   Df Residuals:                      455
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -835.49
Date:                Tue, 12 Sep 2023   Deviance:                       571.34
Time:                        22:37:35   Pearson chi2:                     873.
No. Iterations:                     5   Pseudo R-squ. (CS):           0.007012
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    0.2687      0.397      0.676      0.499      -0.510       1.047
Scope[T.aquifer|transboundary]               0.4388      0.401      1.093      0.274      -0.348       1.225
Longitude                                    0.0004      0.002      0.166      0.868      -0.004       0.005
Longitude:Scope[T.aquifer|transboundary]    -0.0006      0.002     -0.260      0.795      -0.005       0.004
Latitude                                     0.0065      0.009      0.695      0.487      -0.012       0.025
Scope[T.aquifer|transboundary]:Latitude     -0.0070      0.009     -0.736      0.462      -0.026       0.012
Longitude:Latitude                        2.067e-05    2.2e-05      0.940      0.347   -2.24e-05    6.38e-05
============================================================================================================
"""